# Testing the Neural Network

Import libraries required to test the neural network built

In [1]:
import numpy as np
import main
import random as rd

In this case, we are testing the init_layers function. We are checking if the shape of the weight matrices is correct. If the test passes, we print "All tests pass".
First we try a test case to see if the function is working correctly.

In [2]:
from main import init_layers

def test_init_layers():
    W, b = init_layers(3, [3, 2, 2])
    assert W[0].shape == (2, 3)
    assert W[1].shape == (2, 2)
    assert b[0].shape == (3, 1)
    assert b[1].shape == (2, 1)
    assert b[2].shape == (2, 1)

    print("All tests pass")
    
test_init_layers()

Expected number of features:  3
All tests pass


Here it works as expected.
Let's generalize the test case to check if the function is working correctly for all cases.

In [3]:
import random
from main import init_layers

L = random.randint(1, 10)
dims = [random.randint(1, 10) for _ in range(L)]

def test_init_layers():
    W, b = init_layers(L, dims)
    for i in range(L - 1):
        assert W[i].shape == (dims[i + 1], dims[i])
        assert b[i].shape == (dims[i], 1)
    print("All tests pass")
    
test_init_layers()


Expected number of features:  2
All tests pass


The function is working correctly for all cases - random number of layers and random number of neurons in each layer.

Now we can test the feedforward function. We will test the function with a simple test case to see if it is working correctly.

In [4]:
from main import feed_forward, sigmoid

def test_forward_propagation():

    W = [np.array([[1.], [2.]]), np.array([[1., 2.]])]
    b = [np.array([[0]]), np.array([[1.], [2.]]), np.array([[3.]])]

    L=3
    # n = [1, 2, 1] number of neurons
    X = np.array([[1.]])
    
    A, Z, y_hat = feed_forward(L, X, W, b)

    assert len(Z) == len(A)
    assert len(A) == L

    # We calculated by hand the expected results, to check if we obtain the matching values

    assert Z[1][0] == 2
    assert A[1][0] == sigmoid(Z[1][0])

    assert Z[1][1] == 4
    assert A[1][1] == sigmoid(Z[1][1])
    
    assert Z[2] == W[1] @ A[1] + b[2]
    assert sigmoid(Z[2]) == y_hat

    """ print("------------------")
    print(W[1] @ A[1] + b[1])
    print(Z[2])
    print("------------------") """
    print("All tests pass")
    
test_forward_propagation()

Assertion going on
Assertation value ok
(2, 1)
(1, 1)
[[1.]]
(1, 2)
(2, 1)
[[0.88079708]
 [0.98201379]]
All tests pass


This use case does work as expected.
Let's now generalize the test to check if the feed forward function works well for any layer and number of neurons per layer.

Then, we'll generalize the test once again to test the robustness of the function based on the dimension of the input matrix X

In [5]:
import random as rd
from main import feed_forward

def test_forward_propagation_n():

    # Limited to 10 layers - 10 layers probably won't ever be need in our case and we don't have infinite calculation power
    L = rd.randint(3, 10)

    dims = []
    for i in range(L):
        dims.append(rd.randint(1, 5))


    # dims = [rd.randint(1, 10) for _ in range(L)] learn generators

    W, b = init_layers(L, dims)
    assert len(W) + 1 == len(b)

    # number of lines of input matrix = n
    # number of columns = 1 for now - robustness test afterwards
    n1 = dims[0]

    # Need to learn the use of generators
    X_list = []
    for i in range(n1):
        X_list.append([rd.randint(1, 200)])

    X = np.array(X_list)

    print("X:", X)

    A, Z, y_hat = feed_forward(L, X, W, b)

    assert len(Z) == len(A)
    assert len(A) == L
    
    for i in range(1, L):
        assert np.array_equal(Z[i], W[i-1] @ A[i-1] + b[i])     
        assert np.array_equal(A[i], sigmoid(Z[i]))

    i = rd.randint(1, L)
    # The values below are the same. As expected.
    # print(W[i-1])
    # print(A[i-1])
    # print(b[i])
    # print(W[i-1] @ A[i-1] + b[i])
    # print("------------")
    # print(Z[i])                     # After calculating by hand, we can see this value is true
    # print("------------")
    # print("------------")
    print(A)
    print(y_hat)
    

    print("------------------")
    print("All tests pass")

test_forward_propagation_n()

Expected number of features:  5
X: [[134]
 [152]
 [ 39]
 [200]
 [  4]]
Assertion going on


AssertionError: Expected shape of X: number of samples in line, number of features in column

The function is working correctly for all cases - random number of layers and random number of neurons in each layer.

Now we can test the backpropagation function. We will test the function with a simple test case to see if it is working correctly.

In [2]:
import main
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# Initialize layers
nb_layers = 3
dims = [2, 1, 1]
W, b = main.init_layers(nb_layers, dims)

# Input and target
X = np.array([[1, 2]])
y = np.array([[1, 0]])

print("X:", X)
print(X.shape)


Expected number of features:  2
X: [[1 2]]
(1, 2)


In [4]:
# Overfitting test 
from main import train_nn
from main import sigmoid, sigmoid_derivative
from main import binary_cross_entropy, binary_cross_entropy_derivative, mean_squared_error, mean_squared_error_derivative

X = np.array([[1, 2]])
y = np.array([[1]])

print("Number of features in entry data: ", len(X[0]))

# Train the model
epochs, losses = main.train_nn(4, X, y, [2, 2, 2, 1], 0.1, 10000, g=sigmoid, gder=sigmoid_derivative, loss=binary_cross_entropy, lossder=binary_cross_entropy_derivative)

print(losses)
print(epochs)

# Plot the losses
plt.plot(epochs, losses)
plt.xlabel("Epoch i")
plt.ylabel("Loss")
plt.title("Loss function")
plt.show()


Number of features in entry data:  2
[[1 2]]
<class 'numpy.ndarray'>
(1, 2)
Expected number of features:  2
Assertion going on
Assertation value ok
(2, 2)
(2, 1)
[[1]
 [2]]
(2, 2)
(2, 2)
[[0.95131207 0.95131207]
 [0.13439432 0.13439432]]
(1, 2)
(2, 2)
[[0.69413816 0.69413816]
 [0.45167304 0.45167304]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 2)

We seem to have an issue with epoch 1. Let's try to debug the issue.
We will decompose the train function to proceed step by step (ie. epoch by epoch) and see where the issue is.

In [ ]:
import numpy as np
from main import init_layers, feed_forward, backpropagation, update_parameters, sigmoid, sigmoid_derivative, binary_cross_entropy, binary_cross_entropy_derivative

learning_rate = 0.1
nb_layers = 3
nb_neurons = [2, 1, 1]

# epoch 0 - epochs is set to 1 

X = np.array([[1, 2]])
y = np.array([[1]])

print("Number of features in entry data: ", len(X[0]))

nb_samples = len(X)

print(X)
print(type(X))
print(X.shape)

#### Improvement suggestions - automatise the number of neurons in the input layer based on the number of features of the input data
W, b = init_layers(nb_layers, nb_neurons)


A, Z, y_hat = feed_forward(nb_layers, X, W, b, g=sigmoid)
grad_W, grad_b = backpropagation(nb_layers, X, y, W, b, A, Z, y_hat, g=sigmoid, gder=sigmoid_derivative, lossder=binary_cross_entropy_derivative)
W, b = update_parameters(W, b, grad_W, grad_b, learning_rate)

print("W: ", W)
print("b: ", b)



    

Number of features in entry data:  2
[[1 2]]
<class 'numpy.ndarray'>
(1, 2)
Expected number of features:  2
Assertion going on
Assertation value ok
(1, 2)
(2, 1)
[[1]
 [2]]
(1, 1)
(1, 1)
[[0.62520358]]
W:  [array([[-0.72841006,  0.42522004]]), array([[0.60581614]])]
b:  [array([[0.],
       [0.]]), array([[0.40493033]]), array([[-0.09285938]])]


We identified the problem - the issue is with the update of the bias matrices. The list of biaises is as follows:
[(2, 1), (2, 1), (1, 1)]

Since our neural network is of three layers composed with [2, 1, 1] neurons, we expect to have the following list of biases:
[(2, 1), (1, 1), (1, 1)]


In [6]:
print("b: ", b)

for i in range(len(grad_b)-1):
    b[i+1] = b[i+1] - learning_rate * grad_b[i+1]
    
print("b: ", b)

b:  [array([[0.],
       [0.]]), array([[1.52077846]]), array([[-1.1297131]])]
b:  [array([[0.],
       [0.]]), array([[1.93021476]]), array([[-1.28675655]])]


Obviously b[i+1] = b[i+1] - learning_rate * grad_b[i] doesn't work because the dimensions of the matrices before and after calculation are different.

This makes sense because the dimensions of the matrices are different between the layers. We need to update the bias matrices in a different way.